In [ ]:
%pip install lime

In [ ]:
import utils_data as utils
import torch
import torch.nn as nn 
import torch.nn.functional as F
from torchtext.data.utils import get_tokenizer
from torchtext.data import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.data.functional import to_map_style_dataset
from torch.utils.data import DataLoader
from torch.optim import Adam

from tqdm import tqdm
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import gc

from lime import lime_text
from numpy import random

device = "cuda" if torch.cuda.is_available() else "cpu"

### Importation du dataset

In [ ]:
text_train, text_val = utils.get_data_split()
label_train, label_val = utils.get_labels_split()
text_test = utils.get_test()

merged_train = utils.merge_data_labels(text_train, label_train)
merged_val = utils.merge_data_labels(text_val, label_val)

In [ ]:
# Solution pour randomiser la séparation train / val
text_all = utils.get_data()
labels_all = utils.get_labels()
limit = int(len(labels_all) * 0.8) + 1

merged_all = utils.merge_data_labels(text_all, labels_all)
random.shuffle(merged_all)

merged_train, merged_val = merged_all[:limit], merged_all[limit:] # Séparation entre les données de train et de validation (80% - 20%)

Note importante : toute la suite est inspiré de : https://coderzcolumn.com/tutorials/artificial-intelligence/pytorch-rnn-for-text-classification-tasks, ainsi que du cours IFT 6135-A2022.

### Création de tokens à partir des phrases

In [ ]:
tokenizer = get_tokenizer("basic_english")

def build_vocabulary(datasets):
    for dataset in datasets:
        for text in dataset:
            yield tokenizer(text)

vocab = build_vocab_from_iterator(build_vocabulary([text_train, text_val, text_test]), min_freq=1, specials=["<UNK>"])
vocab.set_default_index(vocab["<UNK>"])
len(vocab)

### Préparation des données

In [ ]:
data_train, data_val = to_map_style_dataset(merged_train), to_map_style_dataset(merged_val)
data_test = to_map_style_dataset(text_test)

target_classes = ["negative", "neutral", "positive"]
max_words = 25

def vectorize_batch(batch):
    X, Y = list(zip(*batch))
    X = [vocab(tokenizer(text)) for text in X]
    X = [tokens+([0]* (max_words-len(tokens))) if len(tokens)<max_words else tokens[:max_words] for tokens in X] ## Bringing all samples to max_words length.

    return torch.tensor(X, dtype=torch.int32), torch.tensor(Y, dtype = torch.long)


def vectorize_test_batch(batch):
    X = list(batch)
    X = [vocab(tokenizer(text)) for text in X]
    X = [tokens+([0]* (max_words-len(tokens))) if len(tokens)<max_words else tokens[:max_words] for tokens in X] ## Bringing all samples to max_words length.

    return torch.tensor(X, dtype=torch.int32)

train_loader = DataLoader(data_train, batch_size=1024, collate_fn=vectorize_batch, shuffle=True)
val_loader = DataLoader(data_val, batch_size=1024, collate_fn=vectorize_batch)
test_loader = DataLoader(data_test, batch_size=1024, collate_fn=vectorize_test_batch)


### Définition du RNN

In [ ]:
embed_len = 50
hidden_dim = 50
n_layers = 3
p_dropout = 0.5

class RNNClassifier(nn.Module):
    def __init__(self):
        super(RNNClassifier, self).__init__()
        self.embedding_layer = nn.Embedding(num_embeddings=len(vocab), embedding_dim=embed_len)
        self.rnn = nn.RNN(input_size=embed_len, hidden_size=hidden_dim, num_layers=n_layers, dropout=p_dropout, batch_first=True)
        self.linear = nn.Linear(hidden_dim, len(target_classes))

    def forward(self, X_batch):
        embeddings = self.embedding_layer(X_batch).to(device)
        output, hidden = self.rnn(embeddings)
        return self.linear(output[:,-1])

### Entraînement du RNN

In [ ]:
def evaluate(model, loss_fn, val_loader):
    model.eval()
    with torch.no_grad():
        Y_true, Y_preds, losses = [],[],[]
        for X, Y in val_loader:
            Y_true.append(Y)
            Y = torch.tensor(utils.to_one_hot(Y.numpy(), len(target_classes)))
            X, Y = X.to(device), Y.to(device)
            preds = model(X)
            loss = loss_fn(preds, Y)
            losses.append(loss.item())
            Y_preds.append(preds.argmax(dim=-1))

        Y_true = torch.cat(Y_true)
        Y_preds = torch.cat(Y_preds)

        print("Valid Loss : {:.3f} | Valid Acc : {:.3f}".format(torch.tensor(losses).mean(), accuracy_score(Y_true.detach().cpu().numpy(), Y_preds.detach().cpu().numpy())))


def train(model, loss_fn, optimizer, train_loader, val_loader, epochs=10):
    for i in range(1, epochs+1):
        Y_true, Y_preds, losses = [],[],[]
        for X, Y in tqdm(train_loader):
            Y_true.append(Y)
            Y = torch.tensor(utils.to_one_hot(Y.numpy(), len(target_classes)))
            X, Y = X.to(device), Y.to(device)
            optimizer.zero_grad()
            preds = model(X)
            loss = loss_fn(preds, Y)
            losses.append(loss.item())
            Y_preds.append(preds.argmax(dim=-1))
            loss.backward()
            optimizer.step()

        Y_true = torch.cat(Y_true)
        Y_preds = torch.cat(Y_preds)
            
        print("Epoch {} | Train Loss : {:.3f} | Train acc : {:.3f}".format(i, torch.tensor(losses).mean(), accuracy_score(Y_true.detach().cpu().numpy(), Y_preds.detach().cpu().numpy())))
        evaluate(model, loss_fn, val_loader)
        model.train()

In [ ]:
epochs = 8
learning_rate = 0.001

loss_fn = nn.CrossEntropyLoss()
rnn_classifier = RNNClassifier().to(device)
optimizer = Adam(rnn_classifier.parameters(), lr=learning_rate)

train(rnn_classifier, loss_fn, optimizer, train_loader, val_loader, epochs)

### Prédictions sur l'ensemble de test

In [ ]:
def predict(model, loader):
    Y_preds = []
    for X in tqdm(loader):
        X = X.to(device)
        preds = model(X)
        Y_preds.append(preds.detach().cpu())
    gc.collect()
    Y_preds = torch.cat(Y_preds)

    return F.softmax(Y_preds, dim=-1).argmax(dim=-1).numpy()

Y_preds = predict(rnn_classifier, test_loader)

### Enregistrement des résultats

In [ ]:
utils.save_results(Y_preds, "RNN")

### Explicabilité : Matrice de confusion et LIME

In [ ]:
val_loader_test = DataLoader(text_val, batch_size=1024, collate_fn=vectorize_test_batch)
Y_preds_val = predict(rnn_classifier, val_loader_test)
Y_actual_val = label_val[:, 1]

print("Test Accuracy : {}".format(accuracy_score(Y_actual_val, Y_preds_val)))
print("\nClassification Report : ")
print(classification_report(Y_actual_val, Y_preds_val, target_names=target_classes))
print("\nConfusion Matrix : ")
print(confusion_matrix(Y_actual_val, Y_preds_val))

In [ ]:
X_test_text, Y_test = [], []
for X, Y in merged_val:
    X_test_text.append(X)
    Y_test.append(Y)

explainer = lime_text.LimeTextExplainer(class_names=target_classes, verbose=True)

def make_predictions_lime(X_batch_text):
    X = [vocab(tokenizer(text)) for text in X_batch_text]
    X = [tokens+([0]* (max_words-len(tokens))) if len(tokens)<max_words else tokens[:max_words] for tokens in X] ## Bringing all samples to max_words length.
    logits = rnn_classifier(torch.tensor(X, dtype=torch.int32, device = device))
    preds = F.softmax(logits, dim=-1)
    return preds.detach().cpu().numpy()

idx = int(random.uniform(0, len(Y_test), 1))
X = [vocab(tokenizer(text)) for text in X_test_text[idx:idx+1]]
X = [tokens+([0]* (max_words-len(tokens))) if len(tokens)<max_words else tokens[:max_words] for tokens in X] ## Bringing all samples to max_words length.
preds = rnn_classifier(torch.tensor(X, dtype=torch.int32, device = device))
preds = F.softmax(preds, dim=-1)

explanation = explainer.explain_instance(X_test_text[idx], classifier_fn=make_predictions_lime,
                                         labels=Y_test[idx:idx+1])
explanation.show_in_notebook()
print("Prediction : ", target_classes[preds.argmax()])
print("Actual :     ", target_classes[Y_test[idx]])